In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [65]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [66]:
df = pd.read_csv('dataset_merge.csv')
df = df.set_index("session_id")

df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,2.0,0.0,2,2,2
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,1.0,0.0,0.0,1,1,1
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,3.0,0.0,0.0,0.0,0,0,0
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,17.0,0.0,0.0,0.0,0,0,0
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,0.0,5.0,1.0,3.0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49976,2021-01-03 15:10:27.864,2021-01-03 15:27:03.675,8114,23287,2,3,28,995,497,tarde,24900,2021-01-03 15:28:38.196,2.0,0.0,0.0,0.0,0,0,0
49980,2021-03-31 20:09:31.139,2021-03-31 20:12:58.553,23034,2881,3,4,37,207,69,noche,3057,2021-03-31 20:14:19.965,2.0,1.0,0.0,0.0,0,0,0
49991,2020-08-29 16:59:45.686,2020-08-29 16:59:45.686,7490,7490,1,4,19,0,0,tarde,7590,2020-08-29 17:01:09.711,0.0,1.0,0.0,0.0,1,1,1


### ¿Cuántos tipos de artículos se ven en cada sesión?

Esto nos da una idea valiosa sobre si el clustering nos está dando una información lógica. Si el clustering se ha realizado correctamente, sería normal esperar que en la mayoría de las sesiones sólo se haya visto un tipo (o cluster) de productos. Si el clustering devolviera resultados aleatorios, no habría muchas sesiones en las que sólo se viera un tipo de producto, porque los productos se habrían asignado a los clusters de forma aleatoria, y cada sesión tendría mayoritariamente más de un tipo de producto.

In [67]:
cluster_assign = pd.read_csv('cluster_assign.csv')
cluster_assign = cluster_assign[['item_id2', 'Cluster_pred']]

In [68]:
number_of_clusters = 4

df_clusts = df[[str(i) for i in range(0,number_of_clusters)]]
df_clusts = df_clusts.replace(0, np.nan)
df_clusts["distinct"] = df_clusts.count(1)
item_counts = df_clusts["distinct"].value_counts()
print(item_counts)

1    7519
2    2820
3     701
4     132
Name: distinct, dtype: int64


En este caso, buscamos sesiones en las que se haya buscado más de un artículo. Como se puede observar, la diferencia de sesiones en las que se haya visto solo un tipo de artículo y dos tipos disminuye. Esto reafirma lo que hemos demostrado en el punto anterior, además de asegurar que las sesiones que tienen un único tipo de producto no son únicamente sesiones donde solo se ha visualizado un producto.

In [69]:
df_clusts = df[[str(i) for i in range(0,number_of_clusters)]]
df_clusts = df_clusts.replace(0, np.nan)
df_clusts["sum"] = df_clusts.sum(axis=1)
df_clusts = df_clusts[df_clusts["sum"] >= 2.0]
df_clusts.head()
df_clusts["distinct"] = df_clusts.count(1) - 1
df_clusts.head()
item_counts = df_clusts["distinct"].value_counts().sort_index()
print(item_counts)
df_clusts.head()

1    4097
2    2820
3     701
4     132
Name: distinct, dtype: int64


,0,1,2,3,sum,distinct
session_id,,,,,,
3,NaN,NaN,2.0,NaN,2.0,1
18,3.0,NaN,NaN,NaN,3.0,1
19,17.0,NaN,NaN,NaN,17.0,1
24,NaN,5.0,1.0,3.0,9.0,3
28,NaN,4.0,NaN,NaN,4.0,1


## Dividir entre Train - Test

In [70]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=40)
len(train_set),len(test_set)

(8937, 2235)

## Preparar los datos para el modelo

In [71]:
cat_vars = ["first_prod", "last_prod", "time_first_prod", "most_common_cat", "most_seen_cluster", "first_item_cluster", "last_item_cluster"]
num_vars = ["prod_count", "time_diff", "time_per_prod", "count_num_unique_cat"] + [str(i) for i in range(0,number_of_clusters)]

In [72]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

train_set_num = train_set[num_vars]

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler())
    ])

In [73]:
from sklearn.preprocessing import OneHotEncoder

train_set_cat = train_set[cat_vars]

#Establish values for each category -> otherwise the encoder does not work for the test set
items=cluster_assign.item_id2.to_list()
cats=list(range(1,74))
clusts=list(range(0,number_of_clusters))
times=['día','madrugada','tarde','noche']

cat_pipeline = Pipeline([
        ('one_hot_encoder', OneHotEncoder(categories=[items,items,times,cats,clusts,clusts,clusts], handle_unknown='ignore'))
    ])

In [74]:
from sklearn.compose import ColumnTransformer

num_attribs = train_set_num.columns
cat_attribs = train_set_cat.columns

full_pipeline_train = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

train_set_prepared = full_pipeline_train.fit_transform(train_set)
train_set_prepared

<8937x47479 sparse matrix of type '<class 'numpy.float64'>'
	with 134055 stored elements in Compressed Sparse Row format>

## Selección del modelo y entrenamiento

Aquí vamos comparar los resultados que obtenemos con 5 modelos distintos:

 - SGD Classifier
 - Decision Tree Classifier
 - K-Nearest Neighbors Classifier
 - SVC Classifier
 - Random Forest Classifier

 Para evaluar su rendimiento y escoger así el más adecuado, tendremos en cuenta el tiempo de entrenamiento del modelo y su Median Reciprocal Rank. 

### SGD Classifier

In [75]:
from sklearn.linear_model import SGDClassifier

#sgd_class = SGDClassifier(loss='modified_huber', random_state=42)
#sgd_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(sgd_class, open('models/sgd_model.sav', 'wb'))

### Decision Tree Classifier

In [76]:
from sklearn.tree import DecisionTreeClassifier

#tree_class = DecisionTreeClassifier(random_state=42)
#tree_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(tree_class, open('models/tree_model.sav', 'wb'))

### K-Nearest Neighbors Classifier

In [77]:
from sklearn import neighbors

#n_neighbors = 3
#knn_class = neighbors.KNeighborsClassifier(n_neighbors)
#knn_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(knn_class, open('models/knn_model.sav', 'wb'))

### SVC Classifier

In [78]:
from sklearn import svm

#SVC_class = svm.SVC(kernel='linear', probability=True)
#SVC_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(SVC_class, open('models/svc_model.sav', 'wb'))

### Random Forest Classifier

In [79]:
from sklearn.ensemble import RandomForestClassifier

forest_class = RandomForestClassifier(n_estimators=20, random_state=42)
forest_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(forest_class, open('models/random_model.sav', 'wb'))

RandomForestClassifier(n_estimators=20, random_state=42)

In [80]:
from sklearn.feature_selection import SelectFromModel

#sel = SelectFromModel(RandomForestClassifier(n_estimators = 20))
#sel.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())
#sel.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#sel.get_support()

## Evaluación del modelo

#### Probamos cada modelo utilizando el método de Mean Reciprocal Rank. Es el que se utiliza en el concurso RecSys Challenge. Esto nos ayudará a elegir el mejor modelo. 

In [81]:
test_set_prepared = full_pipeline_train.fit_transform(test_set)

**Cálculo del MRR**

El rango recíproco medio (Mean Reciprocal Rank) es una medida estadística para evaluar cualquier proceso que produzca una lista de posibles respuestas a una muestra de consultas, ordenadas por probabilidad de corrección. El rango recíproco de una respuesta a una consulta es el inverso multiplicativo del rango de la primera respuesta correcta: 1 para el primer puesto, 1⁄2 para el segundo, 1⁄3 para el tercero y así sucesivamente. El rango recíproco medio es la media de los rangos recíprocos de los resultados de una muestra de consultas

$$

MRR = \frac{1}{| Q |} \displaystyle\sum_{i=1}^{| Q |} \frac{1}{rank_i}

$$

donde $ \displaystyle {\text{rank}}_{i} $ hace referencia a la posición de rango del primer documento relevante para la consulta i-ésima.

In [82]:
def mean_reciprocal_rank(model, set_to_test, rr = 0, cont = 0):

    predictions = model.predict_proba(set_to_test)

    pred_df_test = pd.DataFrame(predictions)
    pred_df_test.columns = model.classes_

    pred_df_test["session_id"] = test_set.index
    pred_df_test = pred_df_test.merge(df[["item_purch"]], how='inner', on='session_id')
    pred_df_test = pred_df_test[["session_id", "item_purch"] + list(pred_df_test.columns[:-2])]

    for index, row in pred_df_test.iterrows():
        item_purch_act = int(row.iloc[1])
        row = row.iloc[2:]
        row_sorted = row.sort_values(ascending=False)
        items = row_sorted.index.to_list()
        if item_purch_act in items:
            rank = int(items.index(item_purch_act)) + 1
            if rank <= 100:
                rr += 1/rank
        cont += 1

    mrr = rr/cont
    print("The mean reciprocal rank for the " + str(model) + " is " + str(mrr))

In [83]:
#SVC_class = pickle.load(open('models/svc_model.sav', 'rb'))
#mean_reciprocal_rank(sgd_class, test_set_prepared)
#mean_reciprocal_rank(knn_class, test_set_prepared)
#mean_reciprocal_rank(SVC_class, test_set_prepared)
#mean_reciprocal_rank(tree_class, test_set_prepared)
mean_reciprocal_rank(forest_class, test_set_prepared)

The mean reciprocal rank for the RandomForestClassifier(n_estimators=20, random_state=42) is 0.03170299321335861


Classifier | Number of Sessions | Mean Reciprocal Rank | Training Time |
| --- | --- | --- | --- |
SGD | 11172 | 0.026005918927187604 | 17s 300s |
KNN | 11172 | 0.016126264214551485 | 200ms |
SVC | 11172 | 0.015471577711113993 | 2min 51s 800ms |
Decision Tree | 11172 | 0.010970834404048573 | 17s 900ms |
Random Forest | 11172 | 0.03393014598889391 | 18s 900ms |

Basándonos en los resultados que hemos obtenido para cada clasificador, podemos ver que los que obtuvieron los mejores resultados fueron el SGD Classifier y el Random Forest Classifier. El SGD Classifier y el Random Forest Classifier, si bien tienen un tiempo de entrenamiento muy parecido, difieren de forma notable en el valor del MRR. Es por eso que finalmente se opta por usar el Random Forest Classifier, al tener el MRR más elevado.

## Mejores parámetros

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8], 'max_depth':[3,5,7,10]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]


# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_class, param_grid, cv=5,
                           scoring='neg_root_mean_squared_error',
                           return_train_score=True)
#grid_search.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

In [85]:
print("the best parameters are:")
#print(grid_search.best_params_)

the best parameters are:


## Feature Importance

### Feature importance based on mean decrease in impurity

In [86]:
import time
import numpy as np

start_time = time.time()
importances = forest_class.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest_class.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

Elapsed time to compute the importances: 0.014 seconds


In [87]:
import seaborn as sns

def plot_feature_importance(importance,names,model_type):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

plot_feature_importance(forest_class.feature_importances_,cat_vars+num_vars,'RANDOM FOREST')


ValueError: All arrays must be of the same length

In [ ]:
len(forest_class.feature_importances_)

47479

In [ ]:
importances

array([0.0239366 , 0.0284643 , 0.02924445, ..., 0.00128175, 0.00067719,
       0.00086111])

In [ ]:
train_set_prepared.get_shape()

(17989, 47479)

In [ ]:
import pandas as pd

vars = cat_vars + num_vars
vars = np.array(vars)
forest_importances = pd.Series(importances, index=train_set_prepared.indices)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

ValueError: Length of values (47479) does not match length of index (269835)

## Generate TEST Submission

In [ ]:
df_leader = pd.read_csv('test_leaderboard_sessions.csv')

In [ ]:
features = pd.read_csv('item_features.csv')

leader_session_sorted = df_leader.sort_values(by=['date'])

# First date of session
leader_begin_df = leader_session_sorted.groupby('session_id')['date'].first().rename('start_date')

# First product of session
leader_first_prod=leader_session_sorted.groupby('session_id')['item_id'].first().rename('first_prod')

# Number of products seen in the session
leader_prod_count=leader_session_sorted.groupby('session_id')['item_id'].count().rename('prod_count')

# Last date of the session
leader_end_df = leader_session_sorted.groupby('session_id')['date'].last().rename('end_date')

# Last product of the session
leader_last_prod=leader_session_sorted.groupby('session_id')['item_id'].last().rename('last_prod')

# most common category in these session
leader_join_cat = leader_session_sorted.merge(features, on='item_id', how='left').sort_values(by=['feature_category_id'])
leader_common_cat = leader_join_cat.groupby(['session_id'])['feature_category_id'].value_counts().rename('most_common_cat')
leader_common_cat_df = leader_common_cat.to_frame(name = None).reset_index()
leader_common_cat = leader_common_cat_df.groupby(['session_id'])['feature_category_id'].first().rename('most_common_cat')

# counting number of unique categories for session
leader_count_num_unique_cat = leader_common_cat_df.groupby(['session_id'])['feature_category_id'].count().rename('count_num_unique_cat')


# Concat the datasets
leader_times_df = pd.concat([leader_begin_df, leader_end_df, leader_first_prod, leader_last_prod, 
                      leader_prod_count, leader_common_cat, leader_count_num_unique_cat
                     ],axis=1)

# Proper time format
leader_times_df['start_date']=pd.to_datetime(leader_times_df['start_date'])
leader_times_df['end_date']=pd.to_datetime(leader_times_df['end_date'])

# Calculatind the date difference
leader_times_df['time_diff']=(leader_times_df.end_date-leader_times_df.start_date).astype('timedelta64[s]')

# Calculating the time per product
leader_times_df['time_per_prod']=leader_times_df['time_diff']/leader_times_df['prod_count']

# Adding the time of the day
mask=(leader_times_df.start_date.dt.hour>=0) & (leader_times_df.start_date.dt.hour<7)
leader_times_df.loc[mask,'time_first_prod']='madrugada'
mask=(leader_times_df.start_date.dt.hour>=7) & (leader_times_df.start_date.dt.hour<12)
leader_times_df.loc[mask,'time_first_prod']='día'
mask=(leader_times_df.start_date.dt.hour>=12) & (leader_times_df.start_date.dt.hour<18)
leader_times_df.loc[mask,'time_first_prod']='tarde'
mask=(leader_times_df.start_date.dt.hour>=18) & (leader_times_df.start_date.dt.hour<24)
leader_times_df.loc[mask,'time_first_prod']='noche'

In [ ]:
leader_times_df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod
session_id,,,,,,,,,,
26,2021-06-16 09:53:54.158,2021-06-16 09:53:54.158,19185,19185,1,3,24,0.0,0.000000,día
200,2021-06-25 12:23:40.811,2021-06-25 12:24:50.692,17089,4758,4,30,19,69.0,17.250000,tarde
205,2021-06-11 00:28:07.058,2021-06-11 00:28:07.058,8194,8194,1,3,25,0.0,0.000000,madrugada
495,2021-06-14 22:13:06.741,2021-06-14 22:13:06.741,6853,6853,1,4,20,0.0,0.000000,noche
521,2021-06-19 13:50:03.090,2021-06-19 13:50:03.090,26471,26471,1,3,25,0.0,0.000000,tarde
...,...,...,...,...,...,...,...,...,...,...
4439446,2021-06-11 17:16:12.243,2021-06-11 17:17:11.565,20770,21396,3,4,19,59.0,19.666667,tarde
4439458,2021-06-09 04:52:02.785,2021-06-09 04:54:45.378,11715,26499,2,4,29,162.0,81.000000,madrugada
4439550,2021-06-02 17:42:40.481,2021-06-02 17:42:40.481,19086,19086,1,28,18,0.0,0.000000,tarde


In [ ]:
item_clust = cluster_assign[['item_id2', 'Cluster_pred']].set_index('item_id2')

df_sess_clust_leader=df_leader.merge(item_clust, left_on='item_id', right_index=True)
df_sess_clust_leader

,session_id,item_id,date,Cluster_pred
0,26,19185,2021-06-16 09:53:54.158,1
13240,239993,19185,2021-06-21 11:22:20.096,1
34682,661695,19185,2021-06-27 09:31:21.179,1
36264,699226,19185,2021-06-07 09:58:16.167,1
42810,830528,19185,2021-06-14 15:26:44.282,1
...,...,...,...,...
226231,4382253,7523,2021-06-18 14:05:33.111,0
227011,4397116,6327,2021-06-23 21:59:23.999,0
227515,4408745,10278,2021-06-06 11:23:56.511,2
228684,4428903,27394,2021-06-16 17:03:01.114,1


In [ ]:
df_clust_count_leader = df_sess_clust_leader.groupby(['session_id', 'Cluster_pred'])['session_id'].aggregate('count').unstack().fillna(0)

In [ ]:
df_clust_count_leader["most_seen_cluster"] = df_clust_count_leader.idxmax(axis=1)

In [ ]:
df_leader=pd.concat([leader_times_df, df_clust_count_leader], axis=1)

In [ ]:
df_leader = df_leader.reset_index().merge(item_clust, how='left', left_on='first_prod', right_on=item_clust.index).set_index('session_id')
df_leader = df_leader.rename(columns = {'Cluster_pred':'first_item_cluster'})
df_leader = df_leader.reset_index().merge(item_clust, how='left', left_on='last_prod', right_on=item_clust.index).set_index('session_id')
df_leader = df_leader.rename(columns = {'Cluster_pred':'last_item_cluster'})

In [ ]:
df_leader

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,0,1,2,3,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,
26,2021-06-16 09:53:54.158,2021-06-16 09:53:54.158,19185,19185,1,3,24,0.0,0.000000,día,0.0,1.0,0.0,0.0,1,1,1
200,2021-06-25 12:23:40.811,2021-06-25 12:24:50.692,17089,4758,4,30,19,69.0,17.250000,tarde,4.0,0.0,0.0,0.0,0,0,0
205,2021-06-11 00:28:07.058,2021-06-11 00:28:07.058,8194,8194,1,3,25,0.0,0.000000,madrugada,0.0,1.0,0.0,0.0,1,1,1
495,2021-06-14 22:13:06.741,2021-06-14 22:13:06.741,6853,6853,1,4,20,0.0,0.000000,noche,1.0,0.0,0.0,0.0,0,0,0
521,2021-06-19 13:50:03.090,2021-06-19 13:50:03.090,26471,26471,1,3,25,0.0,0.000000,tarde,0.0,1.0,0.0,0.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439446,2021-06-11 17:16:12.243,2021-06-11 17:17:11.565,20770,21396,3,4,19,59.0,19.666667,tarde,3.0,0.0,0.0,0.0,0,0,0
4439458,2021-06-09 04:52:02.785,2021-06-09 04:54:45.378,11715,26499,2,4,29,162.0,81.000000,madrugada,1.0,1.0,0.0,0.0,0,0,1
4439550,2021-06-02 17:42:40.481,2021-06-02 17:42:40.481,19086,19086,1,28,18,0.0,0.000000,tarde,1.0,0.0,0.0,0.0,0,0,0


In [ ]:
df_leader = df_leader.rename(columns ={0:'0', 1:'1', 2:'2', 3:'3'})

In [ ]:
cat_vars+num_vars

['first_prod',
 'last_prod',
 'time_first_prod',
 'most_common_cat',
 'most_seen_cluster',
 'first_item_cluster',
 'last_item_cluster',
 'prod_count',
 'time_diff',
 'time_per_prod',
 'count_num_unique_cat',
 '0',
 '1',
 '2',
 '3']

In [ ]:
df_leader.columns

Index([          'start_date',             'end_date',           'first_prod',
                  'last_prod',           'prod_count',      'most_common_cat',
       'count_num_unique_cat',            'time_diff',        'time_per_prod',
            'time_first_prod',                      0,                      1,
                            2,                      3,    'most_seen_cluster',
         'first_item_cluster',    'last_item_cluster'],
      dtype='object')

In [ ]:
leader_set_prepared = full_pipeline_train.fit_transform(df_leader)

In [ ]:
predictions = forest_class.predict_proba(leader_set_prepared)
#print(predictions)

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.columns = forest_class.classes_
pred_df

,11,18,26,28,33,44,47,54,55,58,...,28117,28124,28126,28132,28133,28135,28137,28139,28140,28141
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
candidate_items_df = pd.read_csv("candidate_items.csv")
columns = candidate_items_df["item_id"].to_list()
real_columns = [b for b in pred_df.columns if b in columns]

In [ ]:
pred_df = pred_df[real_columns]
pred_df["session_id"] = df_leader.index
pred_df = pred_df[["session_id"] + list(pred_df.columns[:-1])]
pred_df

/var/folders/dc/zzpshy3n5qzdd7ywk2xyx44m0000gn/T/ipykernel_63348/3076393300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df["session_id"] = df_leader.index


,session_id,26,33,54,64,69,70,82,93,95,...,28016,28030,28051,28055,28078,28101,28112,28132,28133,28137
0,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4439446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,4439458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,4439550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,4439653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
f = open('final_submission2.csv', 'w')
f.write("session_id,item_id,rank" + '\n')

for index, row in pred_df.iterrows():
    #print(index,row)
    session_id_act = int(row.iloc[0])
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    items = row_sorted.index.to_list()
    #print(items[0])
    #break
    for i in range(1,101):
        f.write(str(int(session_id_act)) + ',' + str(items[i-1]) + ',' + str(i) + '\n')